In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

In this notebook, we are going to build the model. Before going through this, check EDA of this datase

https://www.kaggle.com/code/naarku30/graduate-admissions-eda

In [43]:
df = pd.read_csv('admission_data.csv')
df.head(2)

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76


There are many regression algorithms, we need to find the best performing model

In [44]:
#Train test split
from sklearn.model_selection import train_test_split

X = df.drop(['Chance of Admit '], axis=1)
y = df['Chance of Admit ']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, shuffle=False)

Here GRE is out of 340, TOEFL is out of 120, University Rating is specified in 5 values. Here, every feature is in differnert dimesnion. So to convert everything into one unit, we use Standard Scaler. So that we can reach the global minimum easily.

In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [46]:
# Saving the scaler 

import pickle
pickle.dump(scaler,open('scaling.pkl','wb'))

In [47]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso,Ridge,BayesianRidge,ElasticNet,HuberRegressor,LinearRegression,LogisticRegression,SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

models = [['DecisionTree',DecisionTreeRegressor()],
           ['Linear Regression', LinearRegression()],
           ['RandomForest',RandomForestRegressor()],
           ['KNeighbours', KNeighborsRegressor(n_neighbors = 2)],
           ['SVM', SVR()],
           ['AdaBoostClassifier', AdaBoostRegressor()],
           ['GradientBoostingClassifier', GradientBoostingRegressor()],
           ['Xgboost', XGBRegressor()],
           ['CatBoost', CatBoostRegressor(logging_level='Silent')],
           ['Lasso', Lasso()],
           ['Ridge', Ridge()],
           ['BayesianRidge', BayesianRidge()],
           ['ElasticNet', ElasticNet()],
           ['HuberRegressor', HuberRegressor()]]

print("Results...")

kpis = []

for name,model in models:
    model = model
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    r2_score = model.score(X_train,y_train)
    kpis.append([name,mae,mse,rmse,r2_score])
    
kpi = pd.DataFrame(kpis,columns=['Model','MAE','MSE','RMSE','R2 Score'])
kpi

Results...


,Model,MAE,MSE,RMSE,R2 Score
0,DecisionTree,0.059200,0.006156,0.078460,1.000000
1,Linear Regression,0.034729,0.002002,0.044743,0.803471
2,RandomForest,0.038543,0.002977,0.054566,0.965251
3,KNeighbours,0.053650,0.004860,0.069712,0.902750
4,SVM,0.047529,0.003748,0.061222,0.772084
5,AdaBoostClassifier,0.038415,0.002694,0.051903,0.797040
6,GradientBoostingClassifier,0.038044,0.002980,0.054588,0.909321
7,Xgboost,0.046966,0.004109,0.064099,0.999775
8,CatBoost,0.039499,0.003112,0.055784,0.976357
9,Lasso,0.112222,0.018292,0.135246,0.000000


In [48]:
kpi.sort_values('RMSE',ascending=True)

,Model,MAE,MSE,RMSE,R2 Score
11,BayesianRidge,0.034782,0.001995,0.044670,0.803371
10,Ridge,0.034734,0.002000,0.044721,0.803466
1,Linear Regression,0.034729,0.002002,0.044743,0.803471
13,HuberRegressor,0.036651,0.002374,0.048720,0.797698
5,AdaBoostClassifier,0.038415,0.002694,0.051903,0.797040
2,RandomForest,0.038543,0.002977,0.054566,0.965251
6,GradientBoostingClassifier,0.038044,0.002980,0.054588,0.909321
8,CatBoost,0.039499,0.003112,0.055784,0.976357
4,SVM,0.047529,0.003748,0.061222,0.772084
7,Xgboost,0.046966,0.004109,0.064099,0.999775


We have scored the test data using various regression models. So, we continue with Linear Regression, Random Forest and Ridge. I only selected these because I have knowledge on these models and they are performing good


In [49]:
#Linear Regression Model

lr_model = LinearRegression()
lr_model.fit(X_train,y_train)
pred = lr_model.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("Root Mean Squared error:", rmse)

Root Mean Squared error: 0.04474312524349167


In [50]:
#Random Forest Regressor Model

rf_model = RandomForestRegressor()
rf_model.fit(X_train,y_train)
pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("Root Mean Squared error:", rmse)

Root Mean Squared error: 0.05281467977750127


In [51]:
#Ridge Model

ridge_model = Ridge()
ridge_model.fit(X_train,y_train)
print("Ridge Model score:", ridge_model.score(X_train, y_train))
pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("Root Mean Squared error:", rmse)

Ridge Model score: 0.8034661029324273
Root Mean Squared error: 0.04472093099187097


In [52]:
#K Fold Cross Validation

from sklearn.model_selection import KFold
model = Ridge()
kfold_validation = KFold(10)

from sklearn.model_selection import cross_val_score
results = cross_val_score(model, X, y, cv=kfold_validation)
print(results)

print("Mean:", np.mean(results))

[0.76314891 0.63403983 0.80196708 0.75237451 0.87030984 0.87650853
 0.81908165 0.81853568 0.91871869 0.87566439]
Mean: 0.8130349102443176


In [53]:
# Saving the ridge model

pickle.dump(ridge_model,open('model.pkl','wb'))

# Predictions

From the EDA we have observed many points. Let's take some assumptions and predict for the new data.
We know, to get higher chances of Admit
* GRE: >310
* TOEFL: >105
* CGPA: >8.5
* University Rating: >=4
* Research: 1

In [23]:
#Test 1
t1 = [319,107, 4, 4, 4, 8.1, 0]

print("Linear Regression prediction: ", lr_model.predict(scaler.transform(np.array(t1).reshape(1,-1))))
print("Random Forest prediction: ", lr_model.predict(scaler.transform(np.array(t1).reshape(1,-1))))
print("Ridge prediction:", ridge_model.predict(scaler.transform(np.array(t1).reshape(1,-1))))

Linear Regression prediction:  [0.68449772]
Random Forest prediction:  [0.68449772]
Ridge prediction: [0.68527023]


In [24]:
#Test2
t2 = [319,107, 4, 4, 4, 8.7, 0]

print("Linear Regression prediction: ", lr_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))
print("Random Forest prediction: ", lr_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))
print("Ridge prediction:", ridge_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))

Linear Regression prediction:  [0.75200375]
Random Forest prediction:  [0.75200375]
Ridge prediction: [0.75210039]


In [25]:
#Test2
t2 = [319,107, 4, 4, 4, 8.7, 1]

print("Linear Regression prediction: ", lr_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))
print("Random Forest prediction: ", lr_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))
print("Ridge prediction:", ridge_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))

Linear Regression prediction:  [0.77703111]
Random Forest prediction:  [0.77703111]
Ridge prediction: [0.77712892]


In [26]:
#Test2
t2 = [319,107, 4, 4, 4, 9.4, 1]

print("Linear Regression prediction: ", lr_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))
print("Random Forest prediction: ", lr_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))
print("Ridge prediction:", ridge_model.predict(scaler.transform(np.array(t2).reshape(1,-1))))

Linear Regression prediction:  [0.85578814]
Random Forest prediction:  [0.85578814]
Ridge prediction: [0.85509744]


Lot of dependency on CGPA

In [27]:
#Test3
t3 = [332,114, 5, 4, 4, 9.4, 1]

print("Linear Regression prediction: ", lr_model.predict(scaler.transform(np.array(t3).reshape(1,-1))))
print("Random Forest prediction: ", lr_model.predict(scaler.transform(np.array(t3).reshape(1,-1))))
print("Ridge prediction:", ridge_model.predict(scaler.transform(np.array(t3).reshape(1,-1))))

Linear Regression prediction:  [0.90677949]
Random Forest prediction:  [0.90677949]
Ridge prediction: [0.90663372]


Applying K - FOLD Cross Validation

In [22]:
#Function for Ridge to test the Cross Validation

def ridge(X_train, X_test, y_train, y_test):
    ridge_model = Ridge()
    ridge_model.fit(X_train,y_train)
    print("Ridge Model score:", ridge_model.score(X_train, y_train))
    pred = ridge_model.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    print("Root Mean Squared error:", rmse)

In [23]:
#Hold Out Validation - Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20,random_state=0)
ridge(X_train, X_test, y_train, y_test)


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20,random_state=1)
ridge(X_train, X_test, y_train, y_test)


#Having different RMSE for different rnadom_state. We need to perform K-Fold


Ridge Model score: 0.8320461610880847
Root Mean Squared error: 0.06394657764105446
Ridge Model score: 0.8214595570552597
Root Mean Squared error: 0.05888978209684229


In [24]:
#K Fold Cross Validation

from sklearn.model_selection import KFold
model = ridge_model
kfold_validation = KFold(10)

from sklearn.model_selection import cross_val_score
results = cross_val_score(model, X, y, cv=kfold_validation)
print(results)

print("Mean:", np.mean(results))

[0.76314891 0.63403983 0.80196708 0.75237451 0.87030984 0.87650853
 0.81908165 0.81853568 0.91871869 0.87566439]
Mean: 0.8130349102443176


We can convey this as the average of accuracy occured on 10 folds. So, the minimum accuracy I have obtained is 0.63 and maximum accuracy I have obtained is 0.91 and average of accuracy is 0.81

In [25]:
#Stratified Cross Validation - apply on classification

from sklearn.model_selection import StratifiedKFold
model = ridge_model
sfold_validation = StratifiedKFold(10)

results = cross_val_score(model, X, y, cv=sfold_validation)
print(results)

print("Mean:", np.mean(results))

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [26]:
#Leave one Out Cross validation

from sklearn.model_selection import LeaveOneOut
leaveoneout = LeaveOneOut()
results = cross_val_score(model, X,y,cv=leaveoneout)
print(results)

print("Mean:", np.mean(results))

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan na

In [28]:
#Shuffle split - multiple iterations of train test split similar to Kfold but based on test size
from sklearn.model_selection import ShuffleSplit
split = ShuffleSplit(n_splits=10, test_size=0.3)
results = cross_val_score(model, X,y, cv=split)
print(results)

print("Mean:", np.mean(results))

[0.77339262 0.83910477 0.81324415 0.82358635 0.82899097 0.77070643
 0.84017666 0.85522962 0.78164414 0.84722017]
Mean: 0.8173295883228029
